In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
# pytorch mlp for regression
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
from sklearn import preprocessing
import pandas as pd

prepro_feat = preprocessing.MinMaxScaler()
prepro_targ = preprocessing.MinMaxScaler()
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path)
        # print(df)
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.X = prepro_feat.fit_transform(self.X)

        # self.X = df.drop(['Oxygen'], axis=1).astype('float32')
        self.y = df.values[:, -1].astype('float32')
        self.y = prepro_targ.fit_transform(self.y.reshape(-1, 1))
        # self.y = Dataframe(df['Oxygen'].astype('float32'))
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X

# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=4, shuffle=True)
    test_dl = DataLoader(test, batch_size=4, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.00001)
    # enumerate epochs
    for epoch in range(25):
        print("Epoch: ",epoch)
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
        print("Loss: ", loss)

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# prepare the data
path = './combinedHeaderlessDropped.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(4)
# train the model
train_model(train_dl, model)
# evaluate the model
mse = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
# make a single prediction (expect class=1)


130201 64129
Epoch:  0
Loss:  tensor(0.1174, grad_fn=<MseLossBackward>)
Epoch:  1
Loss:  tensor(0.0683, grad_fn=<MseLossBackward>)
Epoch:  2
Loss:  tensor(0.0896, grad_fn=<MseLossBackward>)
Epoch:  3
Loss:  tensor(0.0008, grad_fn=<MseLossBackward>)
Epoch:  4
Loss:  tensor(0.0140, grad_fn=<MseLossBackward>)
Epoch:  5
Loss:  tensor(0.0917, grad_fn=<MseLossBackward>)
Epoch:  6
Loss:  tensor(0.0047, grad_fn=<MseLossBackward>)
Epoch:  7
Loss:  tensor(0.1161, grad_fn=<MseLossBackward>)
Epoch:  8
Loss:  tensor(0.0146, grad_fn=<MseLossBackward>)
Epoch:  9
Loss:  tensor(0.0611, grad_fn=<MseLossBackward>)
Epoch:  10
Loss:  tensor(0.0609, grad_fn=<MseLossBackward>)
Epoch:  11
Loss:  tensor(0.0726, grad_fn=<MseLossBackward>)
Epoch:  12
Loss:  tensor(0.0039, grad_fn=<MseLossBackward>)
Epoch:  13
Loss:  tensor(0.0821, grad_fn=<MseLossBackward>)
Epoch:  14
Loss:  tensor(0.1068, grad_fn=<MseLossBackward>)
Epoch:  15
Loss:  tensor(0.0839, grad_fn=<MseLossBackward>)
Epoch:  16
Loss:  tensor(0.0134, grad

In [5]:

row = prepro_feat.transform([[131.3,90,137.85,25.7]])  # Actual = 2323.202
yhat = predict(row, model) * (prepro_targ.data_max_ - prepro_targ.data_min_) + prepro_targ.data_min_
print('Predicted: %.3f' % yhat)



Predicted: 2285.230
